In [ ]:
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt
import pandas as pd
from math import sqrt
import access_br_dwgd as access_br_dwgd
from statsmodels.tsa.stattools import adfuller


In [ ]:
timeseries = access_br_dwgd.recuperar_dados_br_dwgd_com_area(False)
#pegando uma estação
timeseries = timeseries.iloc[:, 50].fillna(0)
train_size = int(len(timeseries) * 0.70)
test_size = len(timeseries) - train_size
train, test = timeseries[:train_size], timeseries[train_size:]
plt.plot(timeseries)
plt.title('Daily Rain in station')
plt.xlabel('Date')
plt.ylabel('mm')
plt.show()


In [ ]:
def ad_test(dataset):
     dftest = adfuller(dataset, autolag = 'AIC')
     print("1. ADF : ",dftest[0])
     print("2. P-Value : ", dftest[1])
     print("3. Num Of Lags : ", dftest[2])
     print("4. Num Of Observations Used For ADF Regression:",      dftest[3])
     print("5. Critical Values :")
     for key, val in dftest[4].items():
         print("\t",key, ": ", val)
ad_test(timeseries)

In [ ]:
plot_acf(timeseries, lags=2)
plot_pacf(timeseries, lags=2)
plt.show()

In [ ]:
#ARIMA AUTO
model = ARIMA(train, order=(2, 1, 1))
model = model.fit()
test_forecast = model.forecast(steps=len(test))
test_forecast_series = pd.Series(test_forecast.values, index=test.index)


# RMSE
rmse = sqrt(mean_squared_error(test, test_forecast))
print('RMSE:', rmse)

# MSE
mse = mean_squared_error(test, test_forecast)
print('MSE:', mse)

# MAE
mae = mean_absolute_error(test, test_forecast)
print('MAE:', mae)


In [ ]:

# Plot
plt.figure(figsize=(14,7))
plt.plot(train, label='Training Data')
plt.plot(test, label='Actual Data', color='orange')
plt.plot(test_forecast_series, label='Forecasted Data', color='green')

plt.title('ARIMA Model Evaluation')
plt.xlabel('Date')
plt.ylabel('mm')
plt.legend()
plt.show()
